In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("sample.csv")
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


# Create a temporary DB

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy import text

temp_db = create_engine('sqlite:///:memory:', echo=True)
data = df.to_sql(name='customTable',con=temp_db)

with temp_db.connect() as conn:
    result = conn.execute(text("Select ,  from customTable ORDER BY  DESC LIMIT 1"))
result.all()    

# Create a prompt

In [ ]:
def create_table_definition_prompt(df):
    """
    This function returns a prompt that informs GPT that we want to work with SQL Tables
    """

    prompt = '''### sqlite SQL table, with its properties:
#
# Sales({})
#
'''.format(",".join(str(x) for x in df.columns))
    
    return prompt

print(create_table_definition_prompt(df))

In [ ]:
def prompt_input():
    nlp_text = input("Enter information you want to obtain: ")
    return nlp_text

In [ ]:
nlp_text = prompt_input()
nlp_text

In [ ]:
def combine_prompts(df, query_prompt):
    definition = create_table_definition_prompt(df)
    query_init_string = f"### A query to answer: {query_prompt}\nSELECT"
    return definition+query_init_string

In [ ]:
combine_prompts(df, nlp_text)

# Connect to OpenAI

In [ ]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base =  os.getenv("OPENAI_API_ENDPOINT") # your endpoint should look like the following https://YOUR_RESOURCE_NAME.openai.azure.com/
openai.api_type = 'azure'
openai.api_version = '2022-12-01' # this may change in the future
deployment_name="OpenAi-Test-text-davinci-003"

response = openai.Completion.create(
  engine=deployment_name,  
  model="text-davinci-003",
  # prompt=combine_prompts(df, nlp_text),
  prompt="Summarize this data ",
  temperature=0,
  max_tokens=150,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  # stop=["#", ";"]
)

In [ ]:
def handle_response(response):
    query = response["choices"][0]["text"]
    if query.startswith(" "):
        query = "Select"+ query
    return query

In [ ]:
print(query)

In [ ]:
with temp_db.connect() as conn:
    result = conn.execute(text(handle_response(response)))

In [ ]:
result.all()